# *(Wang, 2002)* Decision making spiking model

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brainpy/examples/blob/main/decision_making/Wang_2002_decision_making_spiking.ipynb)
[![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/brainpy/examples/blob/main/decision_making/Wang_2002_decision_making_spiking.ipynb)

Implementation of the paper: *Wang, Xiao-Jing. "Probabilistic decision making by slow reverberation in cortical circuits." Neuron 36.5 (2002): 955-968.*

- Author : Chaoming Wang (chao.brain@qq.com)

Please refer to github example folder:

- https://github.com/brainpy/examples/blob/master/decision_making/Wang_2002_decision_making_spiking.py